# Processamento de Linguagem Natural

## Similaridade de Texto

Como computar a similaridade entre duas strings?

In [1]:
import warnings
warnings.filterwarnings("ignore")
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
import codecs

In [2]:
a = 'Refrigerador Brastemp CFR45 20L frostfree'
b = 'Geladeira Brastemp CFR45 20L com desgelo automático'

In [3]:
# Tokens similares
tokensA = a.split()
tokensB = b.split()
set(tokensA).intersection(tokensB)

{'20L', 'Brastemp', 'CFR45'}

In [4]:
similar = len(set(tokensA).intersection(tokensB))
total = len(set(tokensA).union(tokensB))
print ('{} tokens similares de {} tokens: {:0.2f}% de similaridade'.format(similar, total, similar/total*100))

3 tokens similares de 9 tokens: 33.33% de similaridade


In [5]:
!pip install jellyfish

In [6]:
!pip install fuzzywuzzy

In [7]:
!pip install metaphone

  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13918 sha256=61d461c4cef6ceeaf9063d7b35e3519e926bb68082faf417c656ae46da8eda82
  Stored in directory: c:\users\krupc\appdata\local\pip\cache\wheels\b2\9e\d9\26be7687b8fe36cd6cacbec34e825a3dbcd3bae54017cfb385
Successfully built metaphone


In [8]:
# Outras métricas usadas para similaridade de texto
import jellyfish
import fuzzywuzzy
import metaphone

In [9]:
print (metaphone.doublemetaphone('caza'))
print (metaphone.doublemetaphone('casa'))

('KS', '')
('KS', '')


In [10]:
# A "jaro-distance" é uma métrica para comparar strings curtas, como nomes de pessoas
jellyfish.jaro_distance(a,b)

0.6568129284234019

## Outras Possibilidades

* Extrair recursos nomeados para medir a importância de cada token
* Usar algum texto básico para preprocessing (lowecase, stemming, etc)
* Remover palavra-chave
* Peso das palavras usando uma medida de importância (TF / IDF, por exemplo)

## Usando word2vec Para Computar Similaridades entre Vetores
https://radimrehurek.com/gensim/models/word2vec.html

Word2vec é um grupo de modelos relacionados que são usados para produzir word embeddings. Esses modelos são redes neurais artificiais de duas camadas que são treinadas para reconstruir contextos linguísticos de palavras. O Word2vec toma como entrada um grande corpus de texto e produz um espaço vetorial, tipicamente de várias centenas de dimensões, com cada palavra única no corpus sendo atribuída um vetor correspondente no espaço. Os vetores de palavras são posicionados no espaço vetorial de tal forma que as palavras que compartilham contextos comuns no corpus estão localizadas próximas umas das outras no espaço.

O Word2vec foi criado por uma equipe de pesquisadores liderada por Tomas Mikolov no Google. O algoritmo foi posteriormente analisado e explicado por outros pesquisadores. Incorporar vetores criados usando o algoritmo Word2vec tem muitas vantagens em comparação com algoritmos anteriores como Latent Semantic Analysis.

In [11]:
# Leitura do Corpus
import codecs

In [12]:
# Carregando o conteúdo do Corpus para um objeto Python
with codecs.open('corpus.txt', encoding = 'utf8') as fp:
    corpus = fp.read()

In [13]:
type(corpus)

str

In [14]:
from nltk import sent_tokenize, word_tokenize

In [15]:
# Tokenization com NLTK - este processo é demorado!!!
sentences = [[w.lower() for w in word_tokenize(sentence, language = 'portuguese')] for sentence in sent_tokenize(corpus, language = 'portuguese')]

In [16]:
from gensim.models import Word2Vec

In [17]:
?Word2Vec

Init signature:
Word2Vec(
    sentences=None,
    corpus_file=None,
    size=100,
    alpha=0.025,
    window=5,
    min_count=5,
    max_vocab_size=None,
    sample=0.001,
    seed=1,
    workers=3,
    min_alpha=0.0001,
    sg=0,
    hs=0,
    negative=5,
    ns_exponent=0.75,
    cbow_mean=1,
    hashfxn=<built-in function hash>,
    iter=5,
    null_word=0,
    trim_rule=None,
    sorted_vocab=1,
    batch_words=10000,
    compute_loss=False,
    callbacks=(),
    max_final_vocab=None,
)
Docstring:     
Train, use and evaluate neural networks described in https://code.google.com/p/word2vec/.

Once you're finished training a model (=no more updates, only querying)
store and use only the :class:`~gensim.models.keyedvectors.KeyedVectors` instance in `self.wv` to reduce memory.

The model can be stored/loaded via its :meth:`~gensim.models.word2vec.Word2Vec.save` and
:meth:`~gensim.models.word2vec.Word2Vec.load` methods.

The trained word vectors can also be stored/loaded from a format 

In [18]:
# Treinando o modelo
modelo = Word2Vec(sentences, size = 100, window = 5, min_count = 5, workers = 8)
modelo.init_sims(replace = True)

In [56]:
modelo.save("word2vec.model")

In [82]:
modelo.most_similar('automático')

[('automática', 0.8280906677246094),
 ('desligamento', 0.7968020439147949),
 ('timer', 0.7524963021278381),
 ('alarme', 0.732936441898346),
 ('auto', 0.7263692617416382),
 ('indicador', 0.7142494916915894),
 ('programável', 0.6976193785667419),
 ('congelamento', 0.6948739290237427),
 ('operação', 0.6788926124572754),
 ('seletor', 0.6779791712760925)]

In [76]:
tokensA = [t.lower() for t in tokensA]
tokensA = ['refrigerador', 'brastemp', '20l']
vectorsA = sum([modelo[token] for token in tokensA])
#vectorsA = sum([modelo[token] for token in tokensA if token in modelo.raw_vocab])

In [77]:
vectorsA

array([-0.15969187,  0.19366117, -0.30506682,  0.39322537,  0.18298057,
        0.41443396, -0.39909595,  0.0689993 ,  0.09932539, -0.21998245,
        0.2694735 , -0.3245198 , -0.01139234,  0.2737621 , -0.08970599,
        0.05354039, -0.2982077 ,  0.0397533 ,  0.229581  , -0.16394584,
       -0.26207238, -0.1299513 ,  0.02194469,  0.16540036,  0.28656614,
        0.20421222, -0.5016476 , -0.09932723,  0.060597  , -0.23591581,
       -0.12641382, -0.04444879,  0.03247914, -0.1593447 ,  0.12053487,
       -0.48713923,  0.09562235,  0.41305596, -0.3097476 ,  0.03467587,
       -0.24795318, -0.0981178 ,  0.09684897, -0.0270567 , -0.22109652,
       -0.45343944,  0.13131467,  0.17256613,  0.199632  ,  0.25951508,
        0.48055473, -0.21339062, -0.10513137, -0.27228254,  0.01897886,
        0.07127055,  0.37035555,  0.3351263 , -0.02654198, -0.20306945,
       -0.13023072, -0.21700804, -0.24481656,  0.19952905, -0.45863524,
       -0.32931697,  0.3168996 , -0.42859682,  0.26833084,  0.03

In [83]:
tokensB = [t.lower() for t in tokensB]
tokensB = ['geladeira', 'brastemp', '20l', 'com', 'automático']
vectorsB = sum([modelo[token] for token in tokensB])
#vectorsB = sum([modelo[token] for token in tokensB if token in modelo.raw_vocab])

In [84]:
from nltk.cluster.util import cosine_distance
print ('Similaridade: {}'.format(abs(1 - cosine_distance(vectorsA, vectorsB))))

Similaridade: 0.809532592527914
